In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from sklearn.externals import joblib
%matplotlib inline
import seaborn as sns

In [ ]:
#pv = pd.read_csv("../input/previous_application.csv")
cc_bal = pd.read_csv("../input/credit_card_balance.csv")
#X = pd.read_csv("../input/application_train.csv")
#ip = pd.read_csv("../input/installments_payments.csv")

In [ ]:
X['CODE_GENDER'].replace('XNA',np.nan, inplace=True)
X['DAYS_LAST_PHONE_CHANGE'].replace(0, np.nan, inplace=True)
X['DAYS_EMPLOYED'].replace(365243,np.nan, inplace=True)


In [ ]:
X['income_credit_percentage'] = X['AMT_INCOME_TOTAL'] / X['AMT_CREDIT']
X['income_per_child'] = X['AMT_INCOME_TOTAL'] / (1 + X['CNT_CHILDREN'])
X['phone_to_birth_ratio'] = X['DAYS_LAST_PHONE_CHANGE'] / X['DAYS_BIRTH']
X['phone_to_employ_ratio'] = X['DAYS_LAST_PHONE_CHANGE'] / X['DAYS_EMPLOYED']
X['car_to_birth_ratio'] = X['OWN_CAR_AGE'] / X['DAYS_BIRTH']
X['car_to_employ_ratio'] = X['OWN_CAR_AGE'] / X['DAYS_EMPLOYED']
X['long_employment'] = (X['DAYS_EMPLOYED'] < -2000).astype(int)
X['retirement_age'] = (X['DAYS_BIRTH'] < -14000).astype(int)

X['cnt_non_child'] = X['CNT_FAM_MEMBERS'] - X['CNT_CHILDREN']
X['child_to_non_child_ratio'] = X['CNT_CHILDREN'] / X['cnt_non_child']
X['income_per_non_child'] = X['AMT_INCOME_TOTAL'] / X['cnt_non_child']
X['credit_per_person'] = X['AMT_CREDIT'] / X['CNT_FAM_MEMBERS']
X['credit_per_child'] = X['AMT_CREDIT'] / (1 + X['CNT_CHILDREN'])
X['credit_per_non_child'] = X['AMT_CREDIT'] / X['cnt_non_child']

In [ ]:
# External sources
X['external_sources_weighted'] = X.EXT_SOURCE_1 * 2 + X.EXT_SOURCE_2 * 3 + X.EXT_SOURCE_3 * 4

In [ ]:
AGGREGATION_RECIPIES = [
    (['CODE_GENDER', 'NAME_EDUCATION_TYPE'], [('AMT_ANNUITY', 'max'),
                                              ('AMT_CREDIT', 'max'),
                                              ('EXT_SOURCE_1', 'mean'),
                                              ('EXT_SOURCE_2', 'mean'),
                                              ('OWN_CAR_AGE', 'max'),
                                              ('OWN_CAR_AGE', 'sum')]),
    (['CODE_GENDER', 'ORGANIZATION_TYPE'], [('AMT_ANNUITY', 'mean'),
                                            ('AMT_INCOME_TOTAL', 'mean'),
                                            ('DAYS_REGISTRATION', 'mean'),
                                            ('EXT_SOURCE_1', 'mean')]),
    (['CODE_GENDER', 'REG_CITY_NOT_WORK_CITY'], [('AMT_ANNUITY', 'mean'),
                                                 ('CNT_CHILDREN', 'mean'),
                                                 ('DAYS_ID_PUBLISH', 'mean')]),
    (['CODE_GENDER', 'NAME_EDUCATION_TYPE', 'OCCUPATION_TYPE', 'REG_CITY_NOT_WORK_CITY'], [('EXT_SOURCE_1', 'mean'),
                                                                                           ('EXT_SOURCE_2', 'mean')]),
    (['NAME_EDUCATION_TYPE', 'OCCUPATION_TYPE'], [('AMT_CREDIT', 'mean'),
                                                  ('AMT_REQ_CREDIT_BUREAU_YEAR', 'mean'),
                                                  ('APARTMENTS_AVG', 'mean'),
                                                  ('BASEMENTAREA_AVG', 'mean'),
                                                  ('EXT_SOURCE_1', 'mean'),
                                                  ('EXT_SOURCE_2', 'mean'),
                                                  ('EXT_SOURCE_3', 'mean'),
                                                  ('NONLIVINGAREA_AVG', 'mean'),
                                                  ('OWN_CAR_AGE', 'mean'),
                                                  ('YEARS_BUILD_AVG', 'mean')]),
    (['NAME_EDUCATION_TYPE', 'OCCUPATION_TYPE', 'REG_CITY_NOT_WORK_CITY'], [('ELEVATORS_AVG', 'mean'),
                                                                            ('EXT_SOURCE_1', 'mean')]),
    (['OCCUPATION_TYPE'], [('AMT_ANNUITY', 'mean'),
                           ('CNT_CHILDREN', 'mean'),
                           ('CNT_FAM_MEMBERS', 'mean'),
                           ('DAYS_BIRTH', 'mean'),
                           ('DAYS_EMPLOYED', 'mean'),
                           ('DAYS_ID_PUBLISH', 'mean'),
                           ('DAYS_REGISTRATION', 'mean'),
                           ('EXT_SOURCE_1', 'mean'),
                           ('EXT_SOURCE_2', 'mean'),
                           ('EXT_SOURCE_3', 'mean')]),
]

In [ ]:
groupby_aggregate_names = []
for groupby_cols, specs in AGGREGATION_RECIPIES:
    group_object = X.groupby(groupby_cols)
    for select, agg in specs:
        groupby_aggregate_name = '{}_{}_{}'.format('_'.join(groupby_cols), agg, select)
        X = X.merge(group_object[select]
                              .agg(agg)
                              .reset_index()
                              .rename(index=str,
                                      columns={select: groupby_aggregate_name})
                              [groupby_cols + [groupby_aggregate_name]],
                              on=groupby_cols,
                              how='left')
        groupby_aggregate_names.append(groupby_aggregate_name)

In [ ]:
X_agg = X[groupby_aggregate_names + ['SK_ID_CURR']]
X_agg.head()

In [ ]:
pd.merge(X, X_agg, on=['SK_ID_CURR'])

In [ ]:
#X = X.merge(right=X_agg.reset_index(), how='left', on='SK_ID_CURR')
X.head()

In [ ]:
X.head()

In [ ]:
X['CODE_GENDER_NAME_EDUCATION_TYPE_max_AMT_CREDIT_x']

In [ ]:
diff_feature_names = []
for groupby_cols, specs in AGGREGATION_RECIPIES:
    for select, agg in specs:
        if agg in ['mean','median','max','min']:
            groupby_aggregate_name = '{}_{}_{}'.format('_'.join(groupby_cols), agg, select)
            diff_name = '{}_diff'.format(groupby_aggregate_name)
            abs_diff_name = '{}_abs_diff'.format(groupby_aggregate_name)

            X[diff_name] = X[select] - X[groupby_aggregate_name] 
            X[abs_diff_name] = np.abs(X[select] - X[groupby_aggregate_name]) 

            diff_feature_names.append(diff_name)
            diff_feature_names.append(abs_diff_name)

In [ ]:
X_diff = X[diff_feature_names + ['SK_ID_CURR']]

In [ ]:
X_diff.head()

In [ ]:
pd.merge(X, X_diff, on=['SK_ID_CURR'])
X.head()

In [ ]:
X['CODE_GENDER_NAME_EDUCATION_TYPE_max_AMT_ANNUITY']

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn()
%matplotlib inline
import os
import gc
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
import lightgbm as lgb
import xgboost as xgb
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from functools import partial

In [ ]:
pd.set_option('display.max_columns', None) 

In [ ]:
ps = pd.read_csv("../input/POS_CASH_balance.csv")

In [ ]:
ps.head()

In [ ]:
print('Go to dummies')
ps = pd.concat([ps, pd.get_dummies(ps['NAME_CONTRACT_STATUS'])], axis=1)

In [ ]:
features = pd.DataFrame({'SK_ID_CURR': ps['SK_ID_CURR'].unique()})

pos_cash_sorted = ps.sort_values(['SK_ID_CURR', 'MONTHS_BALANCE'])
group_object = pos_cash_sorted.groupby('SK_ID_CURR')['CNT_INSTALMENT_FUTURE'].last().reset_index()
group_object.rename(index=str,
                    columns={'CNT_INSTALMENT_FUTURE': 'pos_cash_remaining_installments'},
                    inplace=True)

features = features.merge(group_object, on=['SK_ID_CURR'], how='left')
features.head()

In [ ]:
ps['is_contract_status_completed'] = ps['NAME_CONTRACT_STATUS'] == 'Completed'
group_object = ps.groupby(['SK_ID_CURR'])['is_contract_status_completed'].sum().reset_index()
group_object.rename(index=str,
                    columns={'is_contract_status_completed': 'pos_cash_completed_contracts'},
                    inplace=True)
features = features.merge(group_object, on=['SK_ID_CURR'], how='left')

ps['pos_cash_paid_late'] = (ps['SK_DPD'] > 0).astype(int)
ps['pos_cash_paid_late_with_tolerance'] = (ps['SK_DPD_DEF'] > 0).astype(int)
groupby = ps.groupby(['SK_ID_CURR'])
features.head()

In [ ]:
 pd.merge(ps, features, on=['SK_ID_CURR'])

In [ ]:
ps.head()

In [ ]:
def  last_k_installment_features(gr, periods):
    gr_ = gr.copy()
    gr_.sort_values(['MONTHS_BALANCE'], ascending=False, inplace=True)

    features = {}
    for period in periods:
        if period > 10e10:
            period_name = 'all_installment_'
            gr_period = gr_.copy()
        else:
            period_name = 'last_{}_'.format(period)
            gr_period = gr_.iloc[:period]

        features = add_features_in_group(features, gr_period, 'pos_cash_paid_late',
                                             ['count', 'mean'],
                                             period_name)
        features = add_features_in_group(features, gr_period, 'pos_cash_paid_late_with_tolerance',
                                             ['count', 'mean'],
                                             period_name)
        features = add_features_in_group(features, gr_period, 'SK_DPD',
                                             ['sum', 'mean', 'max', 'min', 'median'],
                                             period_name)
        features = add_features_in_group(features, gr_period, 'SK_DPD_DEF',
                                             ['sum', 'mean', 'max', 'min','median'],
                                             period_name)
    return features

In [ ]:
def parallel_apply(groups, func, index_name='Index', num_workers=1, chunk_size=100000):
    n_chunks = np.ceil(1.0 * groups.ngroups / chunk_size)
    indeces, features = [], []
    for index_chunk, groups_chunk in chunk_groups(groups, chunk_size, total=n_chunks):
        with mp.pool.Pool(num_workers) as executor:
            features_chunk = executor.map(func, groups_chunk)
        features.extend(features_chunk)
        indeces.extend(index_chunk)

    features = pd.DataFrame(features)
    features.index = indeces
    features.index.name = index_name
    return features

def chunk_groups(groupby_object, chunk_size):
    n_groups = groupby_object.ngroups
    group_chunk, index_chunk = [], []
    for i, (index, df) in enumerate(groupby_object):
        group_chunk.append(df)
        index_chunk.append(index)

        if (i + 1) % chunk_size == 0 or i + 1 == n_groups:
            group_chunk_, index_chunk_ = group_chunk.copy(), index_chunk.copy()
            group_chunk, index_chunk = [], []
            yield index_chunk_, group_chunk_

In [ ]:
#from tqdm import tqdm_notebook as tqdm
import multiprocessing as mp

def add_features_in_group(features, gr_, feature_name, aggs, prefix):
    for agg in aggs:
        if agg == 'sum':
            features['{}{}_sum'.format(prefix, feature_name)] = gr_[feature_name].sum()
        elif agg == 'mean':
            features['{}{}_mean'.format(prefix, feature_name)] = gr_[feature_name].mean()
        elif agg == 'max':
            features['{}{}_max'.format(prefix, feature_name)] = gr_[feature_name].max()
        elif agg == 'min':
            features['{}{}_min'.format(prefix, feature_name)] = gr_[feature_name].min()
        elif agg == 'std':
            features['{}{}_std'.format(prefix, feature_name)] = gr_[feature_name].std()
        elif agg == 'count':
            features['{}{}_count'.format(prefix, feature_name)] = gr_[feature_name].count()
        elif agg == 'skew':
            features['{}{}_skew'.format(prefix, feature_name)] = skew(gr_[feature_name])
        elif agg == 'kurt':
            features['{}{}_kurt'.format(prefix, feature_name)] = kurtosis(gr_[feature_name])
        elif agg == 'iqr':
            features['{}{}_iqr'.format(prefix, feature_name)] = iqr(gr_[feature_name])
        elif agg == 'median':
            features['{}{}_median'.format(prefix, feature_name)] = gr_[feature_name].median()

    return features

In [ ]:
features = pd.DataFrame({'SK_ID_CURR': ps['SK_ID_CURR'].unique()})
func = partial(last_k_installment_features, periods=[1, 10, 50, 10e16])
g = parallel_apply(groupby, func, index_name='SK_ID_CURR', num_workers=10, chunk_size=10000).reset_index()
features = features.merge(g, on='SK_ID_CURR', how='left')
features.head()

In [ ]:
ps = ps.merge(features, on='SK_ID_CURR', how='left')

In [ ]:
ps.head()

In [ ]:
def last_loan_features(gr):
    gr_ = gr.copy()
    gr_.sort_values(['MONTHS_BALANCE'], ascending=False, inplace=True)
    last_installment_id = gr_['SK_ID_PREV'].iloc[0]
    gr_ = gr_[gr_['SK_ID_PREV'] == last_installment_id]

    features={}
    features = add_features_in_group(features, gr_, 'pos_cash_paid_late',
                                         ['count', 'sum', 'mean'],
                                         'last_loan_')
    features = add_features_in_group(features, gr_, 'pos_cash_paid_late_with_tolerance',
                                         ['sum', 'mean'],
                                         'last_loan_')
    features = add_features_in_group(features, gr_, 'SK_DPD',
                                         ['sum', 'mean', 'max', 'min', 'std'],
                                         'last_loan_')
    features = add_features_in_group(features, gr_, 'SK_DPD_DEF',
                                         ['sum', 'mean', 'max', 'min', 'std'],
                                         'last_loan_')
    return features

In [ ]:
features = pd.DataFrame({'SK_ID_CURR': ps['SK_ID_CURR'].unique()})
g = parallel_apply(groupby, last_loan_features, index_name='SK_ID_CURR', num_workers=10, chunk_size=10000).reset_index()
features = features.merge(g, on='SK_ID_CURR', how='left')
features.head()

In [ ]:
ps = ps.merge(features, on='SK_ID_CURR', how='left')

In [ ]:
ps.head()

In [ ]:
def trend_in_last_k_installment_features(gr, periods):
    gr_ = gr.copy()
    gr_.sort_values(['MONTHS_BALANCE'], ascending=False, inplace=True)

    features = {}
    for period in periods:
        gr_period = gr_.iloc[:period]

        features = add_trend_feature(features, gr_period,
                                         'SK_DPD', '{}_period_trend_'.format(period)
                                         )
        features = add_trend_feature(features, gr_period,
                                         'SK_DPD_DEF', '{}_period_trend_'.format(period)
                                         )
    return features

def add_trend_feature(features, gr, feature_name, prefix):
    y = gr[feature_name].values
    try:
        x = np.arange(0, len(y)).reshape(-1, 1)
        lr = LinearRegression()
        lr.fit(x, y)
        trend = lr.coef_[0]
    except:
        trend = np.nan
    features['{}{}'.format(prefix, feature_name)] = trend
    return features

In [ ]:
features = pd.DataFrame({'SK_ID_CURR': ps['SK_ID_CURR'].unique()})
func = partial(trend_in_last_k_installment_features, periods=[1,6,12,30,60])
g = parallel_apply(groupby, func, index_name='SK_ID_CURR', num_workers=10, chunk_size=10000).reset_index()
features = features.merge(g, on='SK_ID_CURR', how='left')
features.head()

In [ ]:
grp = cc_bal.groupby(by = ['SK_ID_CURR'])['AMT_BALANCE'].max().reset_index().rename(index = str, columns ={ 'AMT_BALANCE' : 'MAX_BAL'})
cc_bal = cc_bal.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp 
gc.collect()

grp = cc_bal.groupby(by = ['SK_ID_CURR'])['AMT_BALANCE'].min().reset_index().rename(index = str, columns ={ 'AMT_BALANCE' : 'MIN_BAL'})
cc_bal = cc_bal.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp 
gc.collect()

grp = cc_bal.groupby(by = ['SK_ID_CURR'])['AMT_CREDIT_LIMIT_ACTUAL'].max().reset_index().rename(index = str, columns ={ 'AMT_CREDIT_LIMIT_ACTUAL' : 'MAX_LIM'})
cc_bal = cc_bal.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp 
gc.collect()

grp = cc_bal.groupby(by = ['SK_ID_CURR'])['AMT_CREDIT_LIMIT_ACTUAL'].min().reset_index().rename(index = str, columns ={ 'AMT_CREDIT_LIMIT_ACTUAL' : 'MIN_LIM'})
cc_bal = cc_bal.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp 
gc.collect()

grp = cc_bal.groupby(by = ['SK_ID_CURR'])['AMT_DRAWINGS_ATM_CURRENT'].max().reset_index().rename(index = str, columns ={ 'AMT_DRAWINGS_ATM_CURRENT' : 'MAX_DRW'})
cc_bal = cc_bal.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp 
gc.collect()

grp = cc_bal.groupby(by = ['SK_ID_CURR'])['AMT_DRAWINGS_ATM_CURRENT'].min().reset_index().rename(index = str, columns ={ 'AMT_DRAWINGS_ATM_CURRENT' : 'MIN_DRW'})
cc_bal = cc_bal.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp 
gc.collect()

grp = cc_bal.groupby(by = ['SK_ID_CURR'])['AMT_DRAWINGS_CURRENT'].max().reset_index().rename(index = str, columns ={ 'AMT_DRAWINGS_CURRENT' : 'MAX_C_DRW'})
cc_bal = cc_bal.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp 
gc.collect()

grp = cc_bal.groupby(by = ['SK_ID_CURR'])['AMT_DRAWINGS_CURRENT'].min().reset_index().rename(index = str, columns ={ 'AMT_DRAWINGS_CURRENT' : 'MIN_C_DRW'})
cc_bal = cc_bal.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp 
gc.collect()

grp = cc_bal.groupby(by = ['SK_ID_CURR'])['AMT_DRAWINGS_OTHER_CURRENT'].max().reset_index().rename(index = str, columns ={ 'AMT_DRAWINGS_OTHER_CURRENT' : 'MAX_O_DRW'})
cc_bal = cc_bal.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp 
gc.collect()

grp = cc_bal.groupby(by = ['SK_ID_CURR'])['AMT_DRAWINGS_OTHER_CURRENT'].min().reset_index().rename(index = str, columns ={ 'AMT_DRAWINGS_OTHER_CURRENT' : 'MIN_O_DRW'})
cc_bal = cc_bal.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp 
gc.collect()

grp = cc_bal.groupby(by = ['SK_ID_CURR'])['AMT_DRAWINGS_POS_CURRENT'].max().reset_index().rename(index = str, columns ={ 'AMT_DRAWINGS_POS_CURRENT' : 'MAX_P_DRW'})
cc_bal = cc_bal.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp 
gc.collect()

grp = cc_bal.groupby(by = ['SK_ID_CURR'])['AMT_DRAWINGS_POS_CURRENT'].min().reset_index().rename(index = str, columns ={ 'AMT_DRAWINGS_POS_CURRENT' : 'MIN_P_DRW'})
cc_bal = cc_bal.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp 
gc.collect()

grp = cc_bal.groupby(by = ['SK_ID_CURR'])['AMT_INST_MIN_REGULARITY'].max().reset_index().rename(index = str, columns ={ 'AMT_INST_MIN_REGULARITY' : 'MAX_INST'})
cc_bal = cc_bal.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp 
gc.collect()

grp = cc_bal.groupby(by = ['SK_ID_CURR'])['AMT_INST_MIN_REGULARITY'].min().reset_index().rename(index = str, columns ={ 'AMT_INST_MIN_REGULARITY' : 'MIN_INST'})
cc_bal = cc_bal.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp 
gc.collect()

grp = cc_bal.groupby(by = ['SK_ID_CURR'])['AMT_PAYMENT_CURRENT'].max().reset_index().rename(index = str, columns ={ 'AMT_PAYMENT_CURRENT' : 'MAX_PAYMENT'})
cc_bal = cc_bal.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp 
gc.collect()

grp = cc_bal.groupby(by = ['SK_ID_CURR'])['AMT_PAYMENT_CURRENT'].min().reset_index().rename(index = str, columns ={ 'AMT_PAYMENT_CURRENT' : 'MIN_PAYMENT'})
cc_bal = cc_bal.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp 
gc.collect()

grp = cc_bal.groupby(by = ['SK_ID_CURR'])['AMT_RECEIVABLE_PRINCIPAL'].max().reset_index().rename(index = str, columns ={ 'AMT_RECEIVABLE_PRINCIPAL' : 'MAX_PRIN'})
cc_bal = cc_bal.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp 
gc.collect()

grp = cc_bal.groupby(by = ['SK_ID_CURR'])['AMT_RECEIVABLE_PRINCIPAL'].min().reset_index().rename(index = str, columns ={ 'AMT_RECEIVABLE_PRINCIPAL' : 'MIN_PRIN'})
cc_bal = cc_bal.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp 
gc.collect()

grp = cc_bal.groupby(by = ['SK_ID_CURR'])['AMT_RECIVABLE'].max().reset_index().rename(index = str, columns ={ 'AMT_RECIVABLE' : 'AMT_R_MAX'})
cc_bal = cc_bal.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp 
gc.collect()

grp = cc_bal.groupby(by = ['SK_ID_CURR'])['AMT_RECIVABLE'].min().reset_index().rename(index = str, columns ={ 'AMT_RECIVABLE' : 'AMT_R_MIN'})
cc_bal = cc_bal.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp 
gc.collect()

grp = cc_bal.groupby(by = ['SK_ID_CURR'])['CNT_INSTALMENT_MATURE_CUM'].max().reset_index().rename(index = str, columns ={ 'CNT_INSTALMENT_MATURE_CUM' : 'CNT_INSTL_MAX'})
cc_bal = cc_bal.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp 
gc.collect()

grp = cc_bal.groupby(by = ['SK_ID_CURR'])['CNT_INSTALMENT_MATURE_CUM'].min().reset_index().rename(index = str, columns ={ 'CNT_INSTALMENT_MATURE_CUM' : 'CNT_INSTL_MIN'})
cc_bal = cc_bal.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp 
gc.collect()

In [ ]:
application_train = pd.read_csv('../input/application_train.csv')
application_test = pd.read_csv('../input/application_test.csv')
bureau = pd.read_csv("../input/bureau.csv")
bureau_balance = pd.read_csv("../input/bureau_balance.csv")
cc_bal = pd.read_csv("../input/credit_card_balance.csv")
ip = pd.read_csv("../input/installments_payments.csv")
previous_application = pd.read_csv("../input/previous_application.csv")
ps = pd.read_csv("../input/POS_CASH_balance.csv")

In [ ]:
bureau.head()

In [ ]:
INSTALLMENTS_PAYMENTS_AGGREGATION_RECIPIES = []
for agg in ['mean', 'min', 'max', 'sum', 'var']:
    for select in ['AMT_INSTALMENT',
                   'AMT_PAYMENT',
                   'DAYS_ENTRY_PAYMENT',
                   'DAYS_INSTALMENT',
                   'NUM_INSTALMENT_NUMBER',
                   'NUM_INSTALMENT_VERSION'
                   ]:
        INSTALLMENTS_PAYMENTS_AGGREGATION_RECIPIES.append((select, agg))
INSTALLMENTS_PAYMENTS_AGGREGATION_RECIPIES = [(['SK_ID_CURR'], INSTALLMENTS_PAYMENTS_AGGREGATION_RECIPIES)]

In [ ]:
groupby_aggregate_names = []
for groupby_cols, specs in INSTALLMENTS_PAYMENTS_AGGREGATION_RECIPIES:
    group_object = ip.groupby(groupby_cols)
    for select, agg in specs:
        groupby_aggregate_name = '{}_{}_{}'.format('_'.join(groupby_cols), agg, select)
        groupby_aggregate_names.append(groupby_aggregate_name)

In [ ]:
def last_k_installment_features_with_fractions(gr, periods, period_fractions):
        features = InstallmentPaymentsFeatures.last_k_installment_features(gr, periods)

        for short_period, long_period in period_fractions:
            short_feature_names = get_feature_names_by_period(features, short_period)
            long_feature_names = get_feature_names_by_period(features, long_period)

            for short_feature, long_feature in zip(short_feature_names, long_feature_names):
                old_name_chunk = '_{}_'.format(short_period)
                new_name_chunk = '_{}by{}_fraction_'.format(short_period, long_period)
                fraction_feature_name = short_feature.replace(old_name_chunk, new_name_chunk)
                features[fraction_feature_name] = safe_div(features[short_feature], features[long_feature])
        return features

In [ ]:
cc_bal.head(3)

In [ ]:
grp = previous_application.groupby(by = ['SK_ID_CURR'])['AMT_APPLICATION'].max().reset_index().rename(index = str, columns ={ 'AMT_APPLICATION' : 'DIFF_CREDIT_ANNUITY'})
previous_application = previous_application.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp 
gc.collect()

In [ ]:
previous_application['Isxsell'] = np.where(previous_application['NAME_PRODUCT_TYPE'] == 'x-sell', 1, 0)

In [ ]:
previous_application['IsRefreshed'] = np.where(previous_application['NAME_CLIENT_TYPE'] == 'Refreshed', 1, 0)

In [ ]:
 avg_prev = previous_application.groupby('SK_ID_CURR').mean()

In [ ]:
avg_prev.head()

In [ ]:
previous_application.loc[previous_application['SK_ID_CURR'] == 271877]

In [ ]:
grp = previous_application.groupby(by = ['SK_ID_CURR'])['DAYS_LAST_DUE'].min().reset_index().rename(index = str, columns = {'DAYS_LAST_DUE' : 'MIN_DAYS_LAST_DUE'})
previous_application = previous_application.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp 
gc.collect()

In [ ]:
installments_payments.head()

In [ ]:
ip = pd.read_csv("../input/installments_payments.csv")

In [ ]:
ip['DAYS_INSTALMENT'] = abs(ip['DAYS_INSTALMENT'])
ip['DAYS_ENTRY_PAYMENT'] = abs(ip['DAYS_ENTRY_PAYMENT'])
grp = ip.groupby(by = ['SK_ID_CURR'])['DAYS_INSTALMENT'].sum().reset_index().rename(index = str, columns = {'DAYS_INSTALMENT' : 'DAYS_INSTALMENT_DATE'})
ip = ip.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp
gc.collect()

grp = ip.groupby(by = ['SK_ID_CURR'])['DAYS_ENTRY_PAYMENT'].sum().reset_index().rename(index = str, columns = {'DAYS_ENTRY_PAYMENT' : 'DAYS_PAYMENT_DONE'})
ip = ip.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp
gc.collect()

ip['DIFF'] = (ip['DAYS_PAYMENT_DONE'] - ip['DAYS_INSTALMENT_DATE'])
del ip['DAYS_INSTALMENT_DATE']
del ip['DAYS_PAYMENT_DONE']
gc.collect()

grp = ip.groupby(by = ['SK_ID_CURR'])['DIFF'].max().reset_index().rename(index = str, columns ={ 'DIFF' : 'DIFF_INSTALMENT_PAYMENT_DATE_MAX'})
ip = ip.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp 
gc.collect()

grp = ip.groupby(by = ['SK_ID_CURR'])['DIFF'].min().reset_index().rename(index = str, columns ={ 'DIFF' : 'DIFF_INSTALMENT_PAYMENT_DATE_MIN'})
ip = ip.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp 
gc.collect()
    
del ip['DIFF']
#del ip['DIFF_INSTALMENT_PAYMENT_DATE_x']
#del ip['DIFF_INSTALMENT_PAYMENT_DATE_y']
gc.collect() 

In [ ]:
ps.head()

In [ ]:
grp = ps.groupby(by = ['SK_ID_CURR'])['MONTHS_BALANCE'].min().reset_index().rename(index = str, columns ={ 'MONTHS_BALANCE' : 'MIN_BAL'})
ps = ps.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp 
gc.collect()

In [ ]:
grp = ps.groupby(by = ['SK_ID_CURR'])['MONTHS_BALANCE'].max().reset_index().rename(index = str, columns ={ 'MONTHS_BALANCE' : 'MAX_BAL'})
ps = ps.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp 
gc.collect()

grp = ps.groupby(by = ['SK_ID_CURR'])['CNT_INSTALMENT'].min().reset_index().rename(index = str, columns ={ 'CNT_INSTALMENT' : 'MIN_CNT'})
ps = ps.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp 
gc.collect()

grp = ps.groupby(by = ['SK_ID_CURR'])['CNT_INSTALMENT'].max().reset_index().rename(index = str, columns ={ 'CNT_INSTALMENT' : 'MAX_CNT'})
ps = ps.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp 
gc.collect()

grp = ps.groupby(by = ['SK_ID_CURR'])['CNT_INSTALMENT_FUTURE'].min().reset_index().rename(index = str, columns ={ 'CNT_INSTALMENT_FUTURE' : 'MIN_CNT_FUT'})
ps = ps.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp 
gc.collect()

grp = ps.groupby(by = ['SK_ID_CURR'])['CNT_INSTALMENT_FUTURE'].max().reset_index().rename(index = str, columns ={ 'CNT_INSTALMENT_FUTURE' : 'MAX_CNT_FUT'})
ps = ps.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp 
gc.collect()

In [ ]:
cc_bal.head()

In [ ]:
previous_application['NAME_CLIENT_TYPE'].value_counts()

In [ ]:
train = application_train[['APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG',
                           'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'ELEVATORS_AVG', 'ENTRANCES_AVG',
                           'FLOORSMAX_AVG', 'FLOORSMIN_AVG', 'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG',
                           'LIVINGAREA_AVG','NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAREA_AVG', 'APARTMENTS_MODE', 'BASEMENTAREA_MODE',
                           'YEARS_BEGINEXPLUATATION_MODE', 'YEARS_BUILD_MODE', 'COMMONAREA_MODE', 'ELEVATORS_MODE',
                           'ENTRANCES_MODE', 'FLOORSMAX_MODE', 'FLOORSMIN_MODE', 'LANDAREA_MODE', 'LIVINGAPARTMENTS_MODE', 'TARGET']]

In [ ]:
train = application_train[['ELEVATORS_MEDI','ENTRANCES_MEDI', 'ELEVATORS_MODE', 'ENTRANCES_MODE', 'ELEVATORS_AVG',
                           'ENTRANCES_AVG', 'TARGET']]

In [ ]:
colormap = plt.cm.RdBu
fig = plt.figure(figsize=(20,20))
fig.add_subplot(1,1,1)
plt.title('Correlation of Features', y=1.05, size=8)
sns.heatmap(train.astype(float).corr(),linewidths=0.1,vmax=1.0, 
square=True, cmap=colormap, linecolor='white', annot=True)

In [ ]:
def plot_distribution(feature,color):
    plt.figure(figsize=(10,6))
    plt.title("Distribution of %s" % feature)
    sns.distplot(application_train[feature].dropna(),color=color, kde=True,bins=100)
    plt.show()   

In [ ]:
bureau_balance.head(30)

In [ ]:
bureau_balance['MONTHS_BALANCE'] = bureau_balance['MONTHS_BALANCE'].replace(0, 1)

In [ ]:
bureau_balance['MONTHS_BALANCE'].value_counts()

In [ ]:
replacement = {'C':0, '0':0.1, 'X':0, '1':600, '2': 725, '3':800, '4':900, '5':1000}
bureau_balance['SCORE'] = bureau_balance['STATUS'].apply(lambda x: replacement.get(x))

In [ ]:
bureau_balance = bureau_balance.query('MONTHS_BALANCE != 0')

In [ ]:
bureau_balance.head(25)

In [ ]:
bureau_balance['W_SCORES'] = -(bureau_balance['SCORE']/bureau_balance['MONTHS_BALANCE'])

In [ ]:
avg_buro_bal = bureau_balance.groupby('SK_ID_BUREAU').mean()

In [ ]:
avg_buro_bal.head()

In [ ]:
avg_buro_bal.loc[5001797] #5001797  5715448

In [ ]:
#w = lambda x: np.average(x, weights=-1/bureau_balance.loc[x.index, 'MONTHS_BALANCE'])
#a = {'SCORE': w}
#avg_buro_bal = bureau_balance.groupby('SK_ID_BUREAU')['SCORE'].agg(a)

#wm = lambda x: np.average(x, weights=-1/bureau_balance.loc[x.index, 'MONTHS_BALANCE'])
#avg_bal = bureau_balance.groupby('SK_ID_BUREAU').agg(wm) 

In [ ]:
avg_buro_bal.loc[5715448]

In [ ]:
bureau_balance['MONTHS_BALANCE'].dtype

In [ ]:
buro_bal = pd.read_csv('../input/bureau_balance.csv')
print('Buro bal shape : ', buro_bal.shape)
        
print('transform to dummies')
buro_bal = pd.concat([buro_bal, pd.get_dummies(buro_bal.STATUS, prefix='buro_bal_status')], axis=1).drop('STATUS', axis=1)

print('Counting buros')
buro_counts = buro_bal[['SK_ID_BUREAU', 'MONTHS_BALANCE']].groupby('SK_ID_BUREAU').count()
buro_bal['buro_count'] = buro_bal['SK_ID_BUREAU'].map(buro_counts['MONTHS_BALANCE'])
    
print('averaging buro bal')
avg_buro_bal = buro_bal.groupby('SK_ID_BUREAU').mean()

In [ ]:
buro_bal.head(29)

In [ ]:
avg_buro_bal.head()

In [ ]:
wm = lambda x: np.average(x, weights=-1/credit_card_balance.loc[x.index, 'MONTHS_BALANCE'])
avg_cc_bal = credit_card_balance.groupby('SK_ID_CURR').agg(wm) 
avg_cc_bal.columns = ['cc_bal_' + f_ for f_ in avg_cc_bal.columns]

In [ ]:
POS_CASH_balance.head()

In [ ]:
wm = lambda x: np.average(x, weights=-1/POS_CASH_balance.loc[x.index, 'MONTHS_BALANCE'])
f = {'CNT_INSTALMENT': wm, 'CNT_INSTALMENT_FUTURE': wm, 'SK_DPD': wm, 'SK_DPD_DEF':wm}
cash_avg = POS_CASH_balance.groupby('SK_ID_CURR')['CNT_INSTALMENT','CNT_INSTALMENT_FUTURE',
                                                 'SK_DPD', 'SK_DPD_DEF'].agg(f)

In [ ]:
def missing_data(data):
    total_missing_values = data.isnull().sum().sort_values(ascending=False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    return pd.concat([total_missing_values, percent], axis=1, keys=['Total', 'Percent'])

In [ ]:
def plot_missing_data(data, n=20):
    missing_data(data).iloc[:n, 1].plot(kind='bar',  title='{} largest missing fractions'.format(n))

In [ ]:
#plot_missing_data(application_train, 10)

Since our target variable is 'TARGET', let's do some inspection with it first:

In [ ]:
#application_train['TARGET'].value_counts().plot(kind='pie', figsize=(7,7), title = '% of loan repayers and loan defaulters', autopct='%1.0f%%', 
                                               # explode=(0.3, 0),
                                                #shadow=True)

In [ ]:
def plot_stats(feature,label_rotation=False,horizontal_layout=True):
    temp = application_train[feature].value_counts()
    df1 = pd.DataFrame({feature: temp.index,'Number of contracts': temp.values})

    # Calculate the percentage of target=1 per category value
    cat_perc = application_train[[feature, 'TARGET']].groupby([feature],as_index=False).mean()
    cat_perc.sort_values(by='TARGET', ascending=False, inplace=True)
    
    if(horizontal_layout):
        fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12,6))
    else:
        fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(12,14))
    sns.set_color_codes("pastel")
    s = sns.barplot(ax=ax1, x = feature, y="Number of contracts",data=df1)
    if(label_rotation):
        s.set_xticklabels(s.get_xticklabels(),rotation=90)
    
    s = sns.barplot(ax=ax2, x = feature, y='TARGET', order=cat_perc[feature], data=cat_perc)
    if(label_rotation):
        s.set_xticklabels(s.get_xticklabels(),rotation=90)
    plt.ylabel('Percent of target with value 1 [%]', fontsize=10)
    plt.tick_params(axis='both', which='major', labelsize=10)

    plt.show();

In [ ]:
plot_stats('FONDKAPREMONT_MODE')

In [ ]:
target = application_train['TARGET']

In [ ]:
del application_train['TARGET']

In [ ]:
lb=LabelEncoder()
def DummyEncoding_Cat(df):
    #df=df.copy()
    Cat_Var=df.select_dtypes('object').columns.tolist()
    for col in Cat_Var:
        df[col]=pd.get_dummies(df[col])
    #return df 

In [ ]:
DummyEncoding_Cat(application_train)
DummyEncoding_Cat(application_test)

In [ ]:
def Fill_NA(df):
    #df=df.copy()
    Num_Features=df.select_dtypes(['float64','int64']).columns.tolist()
    #mean = df[Num_Features].mean()
    #df[Num_Features] = df[Num_Features].fillna(mean)
    df[Num_Features]= df[Num_Features].fillna(-999)
    #return df

In [ ]:
#Fill_NA(application_test)
median_ext1 = application_train.groupby(['NAME_INCOME_TYPE'])["EXT_SOURCE_1"].median()

def fillna_ext1(row, median_ext1):
        ext1 = median_ext1.loc[row["NAME_INCOME_TYPE"]]
        return ext1
application_train["EXT_SOURCE_1"] = application_train.apply(lambda row : fillna_ext1(row, median_ext1) if np.isnan(row['EXT_SOURCE_1']) else row['EXT_SOURCE_1'], axis=1)

In [ ]:
#Fill_NA(application_train)
median_ext2 = application_train.groupby(['NAME_INCOME_TYPE'])["EXT_SOURCE_2"].median()

def fillna_ext2(row, median_ext2):
        ext2 = median_ext2.loc[row["NAME_INCOME_TYPE"]]
        return ext2
application_train["EXT_SOURCE_2"] = application_train.apply(lambda row : fillna_ext2(row, median_ext2) if np.isnan(row['EXT_SOURCE_2']) else row['EXT_SOURCE_2'], axis=1)

In [ ]:
median_ext3 = application_train.groupby(['NAME_INCOME_TYPE'])["EXT_SOURCE_3"].median()

def fillna_ext3(row, median_ext3):
        ext3 = median_ext3.loc[row["NAME_INCOME_TYPE"]]
        return ext3
application_train["EXT_SOURCE_3"] = application_train.apply(lambda row : fillna_ext3(row, median_ext3) if np.isnan(row['EXT_SOURCE_3']) else row['EXT_SOURCE_3'], axis=1)

In [ ]:
median_ext3 = application_test.groupby(['NAME_INCOME_TYPE'])["EXT_SOURCE_3"].median()

def fillna_ext3(row, median_ext3):
        ext3 = median_ext3.loc[row["NAME_INCOME_TYPE"]]
        return ext3
application_test["EXT_SOURCE_3"] = application_test.apply(lambda row : fillna_ext3(row, median_ext3) if np.isnan(row['EXT_SOURCE_3']) else row['EXT_SOURCE_3'], axis=1)

In [ ]:
median_ext3 = application_test.groupby(['NAME_INCOME_TYPE'])["EXT_SOURCE_2"].median()

def fillna_ext3(row, median_ext3):
        ext3 = median_ext3.loc[row["NAME_INCOME_TYPE"]]
        return ext3
application_test["EXT_SOURCE_2"] = application_test.apply(lambda row : fillna_ext3(row, median_ext3) if np.isnan(row['EXT_SOURCE_2']) else row['EXT_SOURCE_2'], axis=1)

In [ ]:
median_ext3 = application_test.groupby(['NAME_INCOME_TYPE'])["EXT_SOURCE_1"].median()

def fillna_ext3(row, median_ext3):
        ext3 = median_ext3.loc[row["NAME_INCOME_TYPE"]]
        return ext3
application_test["EXT_SOURCE_1"] = application_test.apply(lambda row : fillna_ext3(row, median_ext3) if np.isnan(row['EXT_SOURCE_1']) else row['EXT_SOURCE_1'], axis=1)

In [ ]:
#col_list = [0, 41, 42, 43, 12, 17, 8, 9, 40, 20, 10, 18, 21, 95, 69, 71, 67, 7, 3, 16, 6, 4, 5, 13, 14, 15, 88, 19, 28]
#application_train = application_train.iloc[:, col_list]
#col1_list = [0, 40, 41, 42, 11, 16, 7, 8, 39, 19, 9, 17, 20, 94, 68, 70, 66, 6, 2, 15, 5, 3, 4, 12, 13, 14, 87, 18, 27]
#application_test = application_test.iloc[:, col1_list]
application_train['EXT_SOURCE_3'].fillna((application_train['EXT_SOURCE_3'].mean()), inplace=True)
application_test['EXT_SOURCE_3'].fillna((application_test['EXT_SOURCE_3'].mean()), inplace=True)

In [ ]:
application_train['EXT_12'] = application_train['EXT_SOURCE_1']*application_train['EXT_SOURCE_2']
application_train['EXT_23'] = application_train['EXT_SOURCE_2']*application_train['EXT_SOURCE_3']
application_train['EXT_31'] = application_train['EXT_SOURCE_1']*application_train['EXT_SOURCE_3']

In [ ]:
col = application_train.loc[:, "EXT_SOURCE_1":"EXT_SOURCE_3"]
application_train['AVG_EXT'] = col.mean(axis=1)

In [ ]:
DummyEncoding_Cat(application_train)

In [ ]:
application_test['EXT_12'] = application_test['EXT_SOURCE_1']*application_test['EXT_SOURCE_2']
application_test['EXT_23'] = application_test['EXT_SOURCE_2']*application_test['EXT_SOURCE_3']
application_test['EXT_31'] = application_test['EXT_SOURCE_1']*application_test['EXT_SOURCE_3']

In [ ]:
col = application_test.loc[:, "EXT_SOURCE_1":"EXT_SOURCE_3"]
application_test['AVG_EXT'] = col.mean(axis=1)

In [ ]:
DummyEncoding_Cat(application_test)

In [ ]:
median_annuity = application_train.groupby(['CODE_GENDER'])['AMT_ANNUITY'].median()

def fillna_ann(row, median_annuity):
        ann = median_annuity.loc[row["CODE_GENDER"]]
        return ann
application_train["AMT_ANNUITY"] = application_train.apply(lambda row : fillna_ann(row, median_annuity) if np.isnan(row['AMT_ANNUITY']) else row['AMT_ANNUITY'], axis=1)

In [ ]:
median_annuity = application_test.groupby(['CODE_GENDER'])['AMT_ANNUITY'].median()

def fillna_ann(row, median_annuity):
        ann = median_annuity.loc[row["CODE_GENDER"]]
        return ann
application_test["AMT_ANNUITY"] = application_test.apply(lambda row : fillna_ann(row, median_annuity) if np.isnan(row['AMT_ANNUITY']) else row['AMT_ANNUITY'], axis=1)

In [ ]:
application_train['BIRTH_CREDIT'] = application_train['AMT_CREDIT']*application_train['DAYS_BIRTH']
application_train['BIRTH_ANNUITY'] = application_train['AMT_ANNUITY']*application_train['DAYS_BIRTH']

application_test['BIRTH_ANNUITY'] = application_test['AMT_ANNUITY']*application_test['DAYS_BIRTH']
application_test['BIRTH_CREDIT'] = application_test['AMT_CREDIT']*application_test['DAYS_BIRTH']

application_train['COST_INCOME'] = application_train['AMT_INCOME_TOTAL'] - application_train['AMT_GOODS_PRICE']
application_test['COST_INCOME'] = application_test['AMT_INCOME_TOTAL'] - application_test['AMT_GOODS_PRICE']

In [ ]:
#application_test['IsSingle'] = np.where(application_test['NAME_FAMILY_STATUS'] == 'Single/not married', 1, 0)
#application_test['IsCivil'] = np.where(application_test['NAME_FAMILY_STATUS'] == 'Civil marriage', 1, 0)
#application_test['IsSecondary'] = np.where(application_test['NAME_EDUCATION_TYPE'] == 'Secondary/secondary special', 1, 0)
#application_test['IsRented'] = np.where(application_test['NAME_HOUSING_TYPE'] == 'Rented apartment', 1, 0)

In [ ]:
#Ratio of debt to credit
application_test['Ratio_of_debt_credit'] = ((application_test['AMT_ANNUITY']+application_test['AMT_INCOME_TOTAL']) - (application_test['AMT_GOODS_PRICE']))/application_test['AMT_CREDIT']
application_train['Ratio_of_debt_credit'] = ((application_train['AMT_ANNUITY']+application_train['AMT_INCOME_TOTAL']) - (application_train['AMT_GOODS_PRICE']))/application_train['AMT_CREDIT']

In [ ]:
# Adding IS_NAN feature for each column
#all_application_df = pd.concat([application_train, application_test], axis=0)
#all_application_is_nan_df = pd.DataFrame()
#for column in all_application_df.columns:
    #if all_application_df[column].isna().sum() == 0:
        #continue
    #all_application_is_nan_df['is_nan_'+column] = all_application_df[column].isna()
    #all_application_is_nan_df['is_nan_'+column] = all_application_is_nan_df['is_nan_'+column].map(lambda v: 1 if v else 0)
#all_application_is_nan_df['target'] = all_application_df['TARGET']
#all_application_is_nan_df = all_application_is_nan_df[pd.notna(all_application_is_nan_df['target'])]

In [ ]:
application_train['Ratio_of_credit_employement'] = application_train['AMT_CREDIT']/application_train['DAYS_EMPLOYED']
application_test['Ratio_of_credit_employement'] = application_test['AMT_CREDIT']/application_test['DAYS_EMPLOYED']
application_test['income_debt_ratio'] = ((application_test['AMT_ANNUITY']+application_test['AMT_INCOME_TOTAL']) - (application_test['AMT_GOODS_PRICE'])) / (application_test['AMT_ANNUITY']+application_test['AMT_INCOME_TOTAL'])
application_train['income_debt_ratio'] = ((application_train['AMT_ANNUITY']+application_train['AMT_INCOME_TOTAL']) - (application_train['AMT_GOODS_PRICE'])) / (application_train['AMT_ANNUITY']+application_train['AMT_INCOME_TOTAL'])

application_train['CreditEmployPdt'] = application_train['AMT_CREDIT'] * application_train['DAYS_EMPLOYED']
application_test['CreditEmployPdt'] = application_test['AMT_CREDIT'] * application_test['DAYS_EMPLOYED']

application_train['DAYS_EMPLOYED_PERC'] = application_train['DAYS_EMPLOYED'] / application_train['DAYS_BIRTH']
application_train['INCOME_CREDIT_PERC'] = application_train['AMT_INCOME_TOTAL'] / application_train['AMT_CREDIT']
application_train['INCOME_PER_PERSON'] = application_train['AMT_INCOME_TOTAL'] / application_train['CNT_FAM_MEMBERS']
application_train['ANNUITY_INCOME_PERC'] = application_train['AMT_ANNUITY'] / application_train['AMT_INCOME_TOTAL']
application_train['ANN_CREDIT_RATIO'] = application_train['AMT_ANNUITY'] / application_train['AMT_CREDIT']

application_test['DAYS_EMPLOYED_PERC'] = application_test['DAYS_EMPLOYED'] / application_test['DAYS_BIRTH']
application_test['INCOME_CREDIT_PERC'] = application_test['AMT_INCOME_TOTAL'] / application_test['AMT_CREDIT']
application_test['INCOME_PER_PERSON'] = application_test['AMT_INCOME_TOTAL'] / application_test['CNT_FAM_MEMBERS']
application_test['ANNUITY_INCOME_PERC'] = application_test['AMT_ANNUITY'] / application_test['AMT_INCOME_TOTAL']
application_test['ANN_CREDIT_RATIO'] = application_test['AMT_ANNUITY'] / application_test['AMT_CREDIT']

application_train['DAYS_UNEMPLOYED_PERC'] = 1 - application_train['DAYS_EMPLOYED_PERC']
application_test['DAYS_UNEMPLOYED_PERC'] = 1 - application_test['DAYS_EMPLOYED_PERC']
application_train['DAYS_UNEMPLOYED'] = application_train['DAYS_BIRTH'] - application_train['DAYS_EMPLOYED']
application_test['DAYS_UNEMPLOYED'] = application_test['DAYS_BIRTH'] - application_test['DAYS_EMPLOYED']

In [ ]:
mean_age_train = application_train.groupby(['CODE_GENDER'])['OWN_CAR_AGE'].mean()

def fillna_carage_train(row, mean_age_train):
    carage_train = mean_age_train.loc[row["CODE_GENDER"]]
    return carage_train
application_train["OWN_CAR_AGE"] = application_train.apply(lambda row : fillna_carage_train(row, mean_age_train) if np.isnan(row['OWN_CAR_AGE']) else row['OWN_CAR_AGE'], axis=1)    

mean_age_test = application_test.groupby(['CODE_GENDER'])['OWN_CAR_AGE'].mean()

def fillna_carage_test(row, mean_age_test):
    carage_test = mean_age_test.loc[row["CODE_GENDER"]]
    return carage_test
application_test["OWN_CAR_AGE"] = application_test.apply(lambda row : fillna_carage_test(row, mean_age_test) if np.isnan(row['OWN_CAR_AGE']) else row['OWN_CAR_AGE'], axis=1) 

application_train['AGE'] = application_train['OWN_CAR_AGE'] + 23
application_test['AGE'] = application_test['OWN_CAR_AGE'] + 23

application_train['AGE'] = application_train['AGE'].astype(int)
application_test['AGE'] = application_test['AGE'].astype(int)

application_train['Age45'] = np.where(application_train['AGE'] >= 45, 1, 0)
application_test['Age45'] = np.where(application_test['AGE'] >= 45, 1, 0)

In [ ]:
#ntrain = application_train.shape[0]
#ntest = application_test.shape[0]
#train = temp_df[:ntrain]
#test = temp_df[ntrain:]
#del application_train['TARGET']

In [ ]:
#sub = application_train.loc[:, 'FLAG_DOCUMENT_2':'FLAG_DOCUMENT_21']
#application_train['PERC_DOCS_SUBMITTED'] = (sub.sum(axis=1))/20
#sub1 = application_test.loc[:, 'FLAG_DOCUMENT_2':'FLAG_DOCUMENT_21']
#application_test['PERC_DOCS_SUBMITTED'] = (sub1.sum(axis=1))/20


sc = StandardScaler()
application_train = sc.fit_transform(application_train)
application_test = sc.transform(application_test)
pca = PCA(n_components=100, random_state=0).fit(application_train)
X_train_pca = pca.transform(application_train)
X_test_pca = pca.transform(application_test)

col_names = []
for i in range(1,101):
    a = "principal_component_" + str(i)
    col_names.append(a)

traindf = pd.DataFrame(data=X_train_pca, columns=col_names)
testdf = pd.DataFrame(data=X_test_pca, columns=col_names)

In [ ]:
#application_train['DAYS_BIRTH'] = abs(application_train['DAYS_BIRTH'])
application_train['AGE'] = application_train['DAYS_BIRTH'] / (365)
#application_test['DAYS_BIRTH'] = abs(application_test['DAYS_BIRTH'])
application_test['AGE'] = application_test['DAYS_BIRTH'] / (365)

In [ ]:
application_train['AGE'] = application_train['AGE'].astype(int)
application_test['AGE'] = application_test['AGE'].astype(int)

In [ ]:
#application_train['0-25'] = np.where(application_train['AGE'] <= 25, 1, 0)
#application_train['25-35'] = np.where((application_train['AGE'] <= 35) & (application_train['AGE'] >= 25), 1, 0)
#application_train['35-50'] = np.where((application_train['AGE'] <= 50) & (application_train['AGE'] >= 35), 1, 0)
#application_train['50-70'] = np.where((application_train['AGE'] <= 70) & (application_train['AGE'] >= 50), 1, 0) 

In [ ]:
#application_test['0-25'] = np.where(application_test['AGE'] <= 25, 1, 0)
#application_test['25-35'] = np.where((application_test['AGE'] <= 35) & (application_test['AGE'] >= 25), 1, 0)
#application_test['35-50'] = np.where((application_test['AGE'] <= 50) & (application_test['AGE'] >= 35), 1, 0)
#application_test['50-70'] = np.where((application_test['AGE'] <= 70) & (application_test['AGE'] >= 50), 1, 0)

In [ ]:
pv.head()

In [ ]:
 #Difference of amount application and amount credit(feature engg for prev application)
grp = pv.groupby(by = ['SK_ID_CURR'])['AMT_CREDIT'].sum().reset_index().rename(index = str, columns = {'AMT_CREDIT' : 'AMT_CREDIT_RECEIVED'})
pv = pv.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp
gc.collect()

grp = pv.groupby(by = ['SK_ID_CURR'])['AMT_APPLICATION'].sum().reset_index().rename(index = str, columns = {'AMT_APPLICATION' : 'AMT_APPLIED_FOR'})
pv = pv.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp
gc.collect()

pv['DIFF'] = (pv['AMT_CREDIT_RECEIVED'] - pv['AMT_APPLIED_FOR'])
del pv['AMT_CREDIT_RECEIVED']
del pv['AMT_APPLIED_FOR']
gc.collect()

grp = pv.groupby(by = ['SK_ID_CURR'])['DIFF'].mean().reset_index().rename(index = str, columns ={ 'DIFF' : 'DIFF_CREDIT_APPLIED'})
pv = pv.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp 
gc.collect()
    
del pv['DIFF']
gc.collect()   

In [ ]:
#Ratio of amount application and amount credit(feature engg for prev application) #new
grp = pv.groupby(by = ['SK_ID_CURR'])['AMT_CREDIT'].sum().reset_index().rename(index = str, columns = {'AMT_CREDIT' : 'AMT_CREDIT_RECEIVED'})
pv = pv.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp
gc.collect()

grp = pv.groupby(by = ['SK_ID_CURR'])['AMT_APPLICATION'].sum().reset_index().rename(index = str, columns = {'AMT_APPLICATION' : 'AMT_APPLIED_FOR'})
pv = pv.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp
gc.collect()

pv['RATIO'] = (pv['AMT_APPLIED_FOR'] / pv['AMT_CREDIT_RECEIVED'])
del pv['AMT_CREDIT_RECEIVED']
del pv['AMT_APPLIED_FOR']
gc.collect()

grp = pv.groupby(by = ['SK_ID_CURR'])['RATIO'].mean().reset_index().rename(index = str, columns ={ 'RATIO' : 'RATIO_CREDIT_APPLIED'})
pv = pv.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp 
gc.collect()

del pv['RATIO']
gc.collect()

In [ ]:
 #Ratio of amount credit and amount annutiy(feature engg for prev application) #new
grp = pv.groupby(by = ['SK_ID_CURR'])['AMT_CREDIT'].sum().reset_index().rename(index = str, columns = {'AMT_CREDIT' : 'AMT_CREDIT_RECEIVED'})
pv = pv.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp
gc.collect()

grp = pv.groupby(by = ['SK_ID_CURR'])['AMT_ANNUITY'].sum().reset_index().rename(index = str, columns = {'AMT_ANNUITY' : 'AMT_ANNUITY_RECEIVED'})
pv = pv.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp
gc.collect()

pv['RATIO'] = (pv['AMT_ANNUITY_RECEIVED'] / pv['AMT_CREDIT_RECEIVED'])
del pv['AMT_CREDIT_RECEIVED']
del pv['AMT_ANNUITY_RECEIVED']
gc.collect()

grp = pv.groupby(by = ['SK_ID_CURR'])['RATIO'].mean().reset_index().rename(index = str, columns ={ 'RATIO' : 'RATIO_ANNUITY_CREDIT'})
pv = pv.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp 
gc.collect()
    
del pv['RATIO']
gc.collect() 

In [ ]:
 #Difference of amount credit and amount annutiy(feature engg for prev application)
grp = pv.groupby(by = ['SK_ID_CURR'])['AMT_CREDIT'].sum().reset_index().rename(index = str, columns = {'AMT_CREDIT' : 'AMT_CREDIT_RECEIVED'})
pv = pv.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp
gc.collect()

grp = pv.groupby(by = ['SK_ID_CURR'])['AMT_ANNUITY'].sum().reset_index().rename(index = str, columns = {'AMT_ANNUITY' : 'AMT_ANNUITY_RECEIVED'})
pv = pv.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp
gc.collect()

pv['DIFF'] = (pv['AMT_CREDIT_RECEIVED'] - pv['AMT_ANNUITY_RECEIVED'])
del pv['AMT_CREDIT_RECEIVED']
del pv['AMT_ANNUITY_RECEIVED']
gc.collect()

grp = pv.groupby(by = ['SK_ID_CURR'])['DIFF'].mean().reset_index().rename(index = str, columns ={ 'DIFF' : 'DIFF_CREDIT_ANNUITY'})
pv = pv.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp 
gc.collect()
    
del pv['DIFF']
gc.collect() 

In [ ]:
#Difference of instalment paid and actual instalment(for installments_payments.csv)
grp = ip.groupby(by = ['SK_ID_CURR'])['AMT_INSTALMENT'].sum().reset_index().rename(index = str, columns = {'AMT_INSTALMENT' : 'AMT_INSTALMENT_ACTUAL'})
ip = ip.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp
gc.collect()

grp = ip.groupby(by = ['SK_ID_CURR'])['AMT_PAYMENT'].sum().reset_index().rename(index = str, columns = {'AMT_PAYMENT' : 'AMT_PAYMENT_DONE'})
ip = ip.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp
gc.collect()

ip['DIFF'] = (ip['AMT_INSTALMENT_ACTUAL'] - ip['AMT_PAYMENT_DONE'])
del ip['AMT_INSTALMENT_ACTUAL']
del ip['AMT_PAYMENT_DONE']
gc.collect()

grp = ip.groupby(by = ['SK_ID_CURR'])['DIFF'].mean().reset_index().rename(index = str, columns ={ 'DIFF' : 'DIFF_INSTALMENT_PAYMENT'})
ip = ip.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp 
gc.collect()
    
del ip['DIFF']
gc.collect() 

In [ ]:
#Ratio of instalment paid and actual instalment(for installments_payments.csv) #new
grp = ip.groupby(by = ['SK_ID_CURR'])['AMT_INSTALMENT'].sum().reset_index().rename(index = str, columns = {'AMT_INSTALMENT' : 'AMT_INSTALMENT_ACTUAL'})
ip = ip.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp
gc.collect()

grp = ip.groupby(by = ['SK_ID_CURR'])['AMT_PAYMENT'].sum().reset_index().rename(index = str, columns = {'AMT_PAYMENT' : 'AMT_PAYMENT_DONE'})
ip = ip.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp
gc.collect()

ip['RATIO'] = (ip['AMT_PAYMENT_DONE'] / ip['AMT_INSTALMENT_ACTUAL'])
del ip['AMT_INSTALMENT_ACTUAL']
del ip['AMT_PAYMENT_DONE']
gc.collect()

grp = ip.groupby(by = ['SK_ID_CURR'])['RATIO'].mean().reset_index().rename(index = str, columns ={ 'RATIO' : 'RATIO_INSTALMENT_PAYMENT'})
ip = ip.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp 
gc.collect()
    
del ip['RATIO']
gc.collect() 

In [ ]:
#Difference between DAYS_INSTALMENT and DAYS_ENTRY_PAYMENT (another column with days_instalment - days_entry payment made by aguair)
ip['DAYS_INSTALMENT'] = abs(ip['DAYS_INSTALMENT'])
ip['DAYS_ENTRY_PAYMENT'] = abs(ip['DAYS_ENTRY_PAYMENT'])
grp = ip.groupby(by = ['SK_ID_CURR'])['DAYS_INSTALMENT'].sum().reset_index().rename(index = str, columns = {'DAYS_INSTALMENT' : 'DAYS_INSTALMENT_DATE'})
ip = ip.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp
gc.collect()

grp = ip.groupby(by = ['SK_ID_CURR'])['DAYS_ENTRY_PAYMENT'].sum().reset_index().rename(index = str, columns = {'DAYS_ENTRY_PAYMENT' : 'DAYS_PAYMENT_DONE'})
ip = ip.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp
gc.collect()

ip['DIFF'] = (ip['DAYS_PAYMENT_DONE'] - ip['DAYS_INSTALMENT_DATE'])
del ip['DAYS_INSTALMENT_DATE']
del ip['DAYS_PAYMENT_DONE']
gc.collect()

grp = ip.groupby(by = ['SK_ID_CURR'])['DIFF'].mean().reset_index().rename(index = str, columns ={ 'DIFF' : 'DIFF_INSTALMENT_PAYMENT_DATE'})
ip = ip.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp 
gc.collect()
    
del ip['DIFF']
gc.collect() 


In [ ]:
ps.head(10)

In [ ]:
#Ratio of unpaid and paid installments
grp = ps.groupby(by = ['SK_ID_CURR'])['CNT_INSTALMENT'].sum().reset_index().rename(index = str, columns = {'CNT_INSTALMENT' : 'CNT_INSTALMENT_PAID'})
ps = ps.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp
gc.collect()

grp = ps.groupby(by = ['SK_ID_CURR'])['CNT_INSTALMENT_FUTURE'].sum().reset_index().rename(index = str, columns = {'CNT_INSTALMENT_FUTURE' : 'CNT_INSTALMENT_TOPAY'})
ps = ps.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp
gc.collect()

ps['RATIO'] = (ps['CNT_INSTALMENT_PAID'] / ps['CNT_INSTALMENT_TOPAY'])
del ps['CNT_INSTALMENT_PAID']
del ps['CNT_INSTALMENT_TOPAY']
gc.collect()

grp = ps.groupby(by = ['SK_ID_CURR'])['RATIO'].mean().reset_index().rename(index = str, columns ={ 'RATIO' : 'RATIO_PAY_TOPAY'})
ps = ps.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp 
gc.collect()
    
del ps['RATIO']
gc.collect

In [ ]:
#Avg number of times DPD has occured
def f(DPD):
    
    # DPD is a series of values of SK_DPD for each of the groupby combination 
    # We convert it to a list to get the number of SK_DPD values NOT EQUALS ZERO
    x = DPD.tolist()
    c = 0
    for i,j in enumerate(x):
        if j != 0:
            c += 1
    
    return c 
        
grp = ps.groupby(by = ['SK_ID_CURR', 'SK_ID_PREV']).apply(lambda x: f(x.SK_DPD)).reset_index().rename(index = str, columns = {0: 'NO_DPD'})
grp1 = grp.groupby(by = ['SK_ID_CURR'])['NO_DPD'].mean().reset_index().rename(index = str, columns = {'NO_DPD' : 'DPD_COUNT'})

ps = ps.merge(grp1, on = ['SK_ID_CURR'], how = 'left')
del grp1
del grp 
gc.collect()
    
#Average of days past dues per customer
grp = ps.groupby(by= ['SK_ID_CURR'])['SK_DPD'].mean().reset_index().rename(index = str, columns = {'SK_DPD': 'AVG_DPD'})
ps = ps.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp 
gc.collect()

In [ ]:
#Average of days past dues with tolerance
grp = ps.groupby(by= ['SK_ID_CURR'])['SK_DPD_DEF'].mean().reset_index().rename(index = str, columns = {'SK_DPD_DEF': 'AVG_DPD_DEF'})
ps = ps.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp 
gc.collect()

In [ ]:
ps['NAME_CONTRACT_STATUS'].values

In [ ]:
app_train['missing_percent'] = app_train.isnull().sum(axis=1) / 122

In [ ]:
app_train['missing_percent']

In [ ]:
def train_model(data_, test_, y_, folds_):

    oof_preds = np.zeros(data_.shape[0])
    sub_preds = np.zeros(test_.shape[0])
    
    feature_importance_df = pd.DataFrame()
    
    feats = [f for f in data_.columns if f not in ['SK_ID_CURR']]
    
    for n_fold, (trn_idx, val_idx) in enumerate(folds_.split(data_)):
        trn_x, trn_y = data_[feats].iloc[trn_idx], y_.iloc[trn_idx]
        val_x, val_y = data_[feats].iloc[val_idx], y_.iloc[val_idx]
        
        clf = LGBMClassifier(
            n_estimators=10000,
            learning_rate=0.03,
            num_leaves=34,
            colsample_bytree=0.9497036,
            subsample=0.8715623,
            max_depth=8,
            boosting_type='goss',
            bagging_freq=0,
            reg_alpha=0.041545473,
            reg_lambda=0.0735294,
            min_split_gain=0.0222415,
            min_child_weight=39.3259775,
            silent=-1,
            verbose=-1,
        )
        
        clf.fit(trn_x, trn_y, 
                eval_set= [(trn_x, trn_y), (val_x, val_y)], 
                eval_metric='auc', verbose=100, early_stopping_rounds=100  
               )
        
        oof_preds[val_idx] = clf.predict_proba(val_x, num_iteration=clf.best_iteration_)[:, 1]
        sub_preds += clf.predict_proba(test_[feats], num_iteration=clf.best_iteration_)[:, 1] / folds_.n_splits
        
        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = clf.feature_importances_
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        
        print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(val_y, oof_preds[val_idx])))
        del clf, trn_x, trn_y, val_x, val_y
        gc.collect()
        
    print('Full AUC score %.6f' % roc_auc_score(y_, oof_preds)) 
    
    test_['TARGET'] = sub_preds

    return oof_preds, test_[['SK_ID_CURR', 'TARGET']], feature_importance_df  



In [ ]:
if __name__ == '__main__':
    gc.enable()
    # Build model inputs
    #application_train, application_test, y = build_model_input()
    # Create Folds
    folds = KFold(n_splits=5, shuffle=True, random_state=546789)
    # Train model and get oof and test predictions
    oof_preds, test_preds, importances = train_model(application_train, application_test, target, folds)
    # Save test predictions
    test_preds.to_csv('first_submission.csv', index=False)

In [ ]:
pv.shape

In [ ]:
pv.head()

In [ ]:
#Number of types of past loans in previous application per customer
grp = pv[['SK_ID_CURR', 'NAME_CONTRACT_TYPE']].groupby(by = ['SK_ID_CURR'])['NAME_CONTRACT_TYPE'].nunique().reset_index().rename(index=str, columns={'NAME_CONTRACT_TYPE': 'LOAN_TYPE'})
pv = pv.merge(grp, on = ['SK_ID_CURR'], how = 'left')
print(pv.shape)
del grp

In [ ]:
#Profit remaining after subtracting goods price from credit and annuity
grp = pv.groupby(by = ['SK_ID_CURR'])['AMT_CREDIT'].sum().reset_index().rename(index = str, columns = {'AMT_CREDIT' : 'AMT_CREDIT_RECEIVED'})
pv = pv.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp
gc.collect()

grp = pv.groupby(by = ['SK_ID_CURR'])['AMT_ANNUITY'].sum().reset_index().rename(index = str, columns = {'AMT_ANNUITY' : 'AMT_ANNUITY_RECEIVED'})
pv = pv.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp
gc.collect()

grp = pv.groupby(by = ['SK_ID_CURR'])['AMT_GOODS_PRICE'].sum().reset_index().rename(index = str, columns = {'AMT_GOODS_PRICE' : 'PRICE_OF_GOODS'})
pv = pv.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp
gc.collect()

pv['PROF'] = ((pv['AMT_CREDIT_RECEIVED'] + pv['AMT_ANNUITY_RECEIVED'])-(pv['PRICE_OF_GOODS']))/(pv['AMT_CREDIT_RECEIVED'] + pv['AMT_ANNUITY_RECEIVED'])
del pv['AMT_CREDIT_RECEIVED']
del pv['AMT_ANNUITY_RECEIVED']
del pv['PRICE_OF_GOODS']
gc.collect()

grp = pv.groupby(by = ['SK_ID_CURR'])['PROF'].mean().reset_index().rename(index = str, columns ={ 'PROF' : 'DEBT_INCOME_RATIO'})
pv = pv.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp 
gc.collect()
    
del pv['PROF']
gc.collect() 


In [ ]:
pv.head(10)

In [ ]:
#How many previous applications of our client was rejected, approved, denied etc
grp = pv[['SK_ID_CURR', 'NAME_CONTRACT_STATUS']].groupby(by = ['SK_ID_CURR'])['NAME_CONTRACT_STATUS'].nunique().reset_index().rename(index=str, columns={'NAME_CONTRACT_STATUS': 'STATUS?'})
pv = pv.merge(grp, on = ['SK_ID_CURR'], how = 'left')
print(pv.shape)
del grp

In [ ]:
#Lets check how many loans were taken for cash, POS, car etc
grp = pv[['SK_ID_CURR', 'NAME_PORTFOLIO']].groupby(by = ['SK_ID_CURR'])['NAME_PORTFOLIO'].nunique().reset_index().rename(index=str, columns={'NAME_PORTFOLIO': 'LOAN_FOR?'})
pv = pv.merge(grp, on = ['SK_ID_CURR'], how = 'left')
print(pv.shape)
del grp

In [ ]:
pv['NFLAG_INSURED_ON_APPROVAL'] = pv['NFLAG_INSURED_ON_APPROVAL'].fillna(0)

In [ ]:
#Now lets check for how many of the previous applications was the applicant insured
grp = pv[['SK_ID_CURR', 'NFLAG_INSURED_ON_APPROVAL']].groupby(by = ['SK_ID_CURR'])['NFLAG_INSURED_ON_APPROVAL'].nunique().reset_index().rename(index=str, columns={'NFLAG_INSURED_ON_APPROVAL': 'INSURED?'})
pv = pv.merge(grp, on = ['SK_ID_CURR'], how = 'left')
print(pv.shape)
del grp

In [ ]:
#Let's check the interest grouping in previous applications per client
grp = pv[['SK_ID_CURR', 'NAME_YIELD_GROUP']].groupby(by = ['SK_ID_CURR'])['NAME_YIELD_GROUP'].nunique().reset_index().rename(index=str, columns={'NAME_YIELD_GROUP': 'INT_GROUP'})
pv = pv.merge(grp, on = ['SK_ID_CURR'], how = 'left')
print(pv.shape)
del grp

In [ ]:
#counting the various diff types of pdt combinations for every applicant's previous loan
grp = pv[['SK_ID_CURR', 'PRODUCT_COMBINATION']].groupby(by = ['SK_ID_CURR'])['PRODUCT_COMBINATION'].nunique().reset_index().rename(index=str, columns={'PRODUCT_COMBINATION': 'COMBINATION_COUNT'})
pv = pv.merge(grp, on = ['SK_ID_CURR'], how = 'left')
print(pv.shape)
del grp

In [ ]:
#counting the no. of various different channels through which the applicants acquired the previous loan
grp = pv[['SK_ID_CURR', 'CHANNEL_TYPE']].groupby(by = ['SK_ID_CURR'])['CHANNEL_TYPE'].nunique().reset_index().rename(index=str, columns={'CHANNEL_TYPE': 'CHANNEL?'})
pv = pv.merge(grp, on = ['SK_ID_CURR'], how = 'left')
print(pv.shape)
del grp

In [ ]:
cc_bal.head(10)

In [ ]:
print('Go to dummies')
cc_bal = pd.concat([cc_bal, pd.get_dummies(cc_bal['NAME_CONTRACT_STATUS'], prefix='cc_bal_status_')], axis=1)
nb_prevs = cc_bal[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()
cc_bal['SK_ID_PREV'] = cc_bal['SK_ID_CURR'].map(nb_prevs['SK_ID_PREV'])
    
print('Compute average')
#wm = lambda x: np.average(x, weights=-1/cc_bal.loc[x.index, 'MONTHS_BALANCE'])
avg_cc_bal = cc_bal.groupby('SK_ID_CURR').mean()
avg_cc_bal.columns = ['cc_bal_' + f_ for f_ in avg_cc_bal.columns]

In [ ]:
app_train = app_train.merge(right=avg_cc_bal.reset_index(), how='left', on='SK_ID_CURR')

In [ ]:
app_train.head(10)

In [ ]:
application_train['TOTAL_NO_ENQUIRY'] = application_train['AMT_REQ_CREDIT_BUREAU_HOUR'] + application_train['AMT_REQ_CREDIT_BUREAU_DAY'] + application_train['AMT_REQ_CREDIT_BUREAU_WEEK'] + application_train['AMT_REQ_CREDIT_BUREAU_MON'] + application_train['AMT_REQ_CREDIT_BUREAU_QRT'] + application_train['AMT_REQ_CREDIT_BUREAU_YEAR']
application_test['TOTAL_NO_ENQUIRY'] = application_test['AMT_REQ_CREDIT_BUREAU_HOUR'] + application_test['AMT_REQ_CREDIT_BUREAU_DAY'] + application_test['AMT_REQ_CREDIT_BUREAU_WEEK'] + application_test['AMT_REQ_CREDIT_BUREAU_MON'] + application_test['AMT_REQ_CREDIT_BUREAU_QRT'] + application_test['AMT_REQ_CREDIT_BUREAU_YEAR']                                        
#this one is new                                        

In [ ]:
app_train['cc_bal_INCOME_RATIO'] = app_train['cc_bal_AMT_BALANCE'] / app_train['AMT_INCOME_TOTAL']

In [ ]:
app_train['cc_payment_INCOME_RATIO'] = app_train['cc_bal_AMT_DRAWINGS_ATM_CURRENT'] / app_train['AMT_INCOME_TOTAL']

In [ ]:
application_train['TOT_SOC_OBS_DPD'] = application_train['OBS_30_CNT_SOCIAL_CIRCLE'] + application_train['OBS_60_CNT_SOCIAL_CIRCLE']
application_test['TOT_SOC_OBS_DPD'] = application_test['OBS_30_CNT_SOCIAL_CIRCLE'] + application_test['OBS_60_CNT_SOCIAL_CIRCLE']

In [ ]:
application_train['TOT_SOC_DEF_DPD'] = application_train['DEF_30_CNT_SOCIAL_CIRCLE'] + application_train['DEF_60_CNT_SOCIAL_CIRCLE']
application_test['TOT_SOC_DEF_DPD'] = application_test['DEF_30_CNT_SOCIAL_CIRCLE'] + application_test['DEF_60_CNT_SOCIAL_CIRCLE']

In [ ]:
application_train['CAR_AGE_RATIO'] = application_train['OWN_CAR_AGE'] / abs(application_train['AGE'].astype(int))
application_test['CAR_AGE_RATIO'] = application_test['OWN_CAR_AGE'] / abs(application_test['AGE'].astype(int))

In [ ]:
application_train['ANNUITY_LENGTH'] = application_train['AMT_CREDIT'] / application_train['AMT_ANNUITY']
application_test['ANNUITY_LENGTH'] = application_test['AMT_CREDIT'] / application_test['AMT_ANNUITY']

application_train['CHILDREN_RATIO'] = application_train['CNT_CHILDREN'] / application_train['CNT_FAM_MEMBERS']
application_test['CHILDREN_RATIO'] = application_test['CNT_CHILDREN'] / application_test['CNT_FAM_MEMBERS']

application_train['ANNUITY_EMPLOYED_RATIO'] = application_train['ANNUITY_LENGTH'] / application_train['DAYS_EMPLOYED']
application_test['ANNUITY_EMPLOYED_RATIO'] = application_test['ANNUITY_LENGTH'] / application_test['DAYS_EMPLOYED']

In [ ]:
#calculating the total no of documents submitted
application_train['TOTAL_DOCS_SUBMITTED'] = application_train.loc[:, application_train.columns.str.contains('FLAG_DOCUMENT')].sum(axis=1)
application_test['TOTAL_DOCS_SUBMITTED'] = application_test.loc[:, application_test.columns.str.contains('FLAG_DOCUMENT')].sum(axis=1)

In [ ]:
cc_bal.head(10)

In [ ]:
#We are going to check the balance and subtract/take ratio of the total drawing from the balance for previous credit
grp = cc_bal.groupby(by = ['SK_ID_CURR'])['AMT_DRAWINGS_ATM_CURRENT'].sum().reset_index().rename(index = str, columns = {'AMT_DRAWINGS_ATM_CURRENT' : 'DRAWINGS_ATM'})
cc_bal = cc_bal.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp
gc.collect()

grp = cc_bal.groupby(by = ['SK_ID_CURR'])['AMT_DRAWINGS_CURRENT'].sum().reset_index().rename(index = str, columns = {'AMT_DRAWINGS_CURRENT' : 'DRAWINGS_CURRENT'})
cc_bal = cc_bal.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp
gc.collect()

grp = cc_bal.groupby(by = ['SK_ID_CURR'])['AMT_DRAWINGS_OTHER_CURRENT'].sum().reset_index().rename(index = str, columns = {'AMT_DRAWINGS_OTHER_CURRENT' : 'DRAWINGS_OTHER'})
cc_bal = cc_bal.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp
gc.collect()

grp = cc_bal.groupby(by = ['SK_ID_CURR'])['AMT_DRAWINGS_POS_CURRENT'].sum().reset_index().rename(index = str, columns = {'AMT_DRAWINGS_POS_CURRENT' : 'DRAWINGS_POS'})
cc_bal = cc_bal.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp
gc.collect()

cc_bal['total_drawings'] = cc_bal['DRAWINGS_ATM'] + cc_bal['DRAWINGS_CURRENT'] + cc_bal['DRAWINGS_OTHER'] + cc_bal['DRAWINGS_POS']

grp = cc_bal.groupby(by = ['SK_ID_CURR'])['AMT_BALANCE'].sum().reset_index().rename(index = str, columns = {'AMT_BALANCE' : 'BALANCE_LEFT'})
cc_bal = cc_bal.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp
gc.collect()

cc_bal['BALANCE_DRAWINGS_RATIO'] = cc_bal['BALANCE_LEFT'] / cc_bal['total_drawings']
cc_bal['BALANCE_DRAWINGS_DIFF'] = cc_bal['BALANCE_LEFT'] - cc_bal['total_drawings']

del cc_bal['BALANCE_LEFT']
del cc_bal['total_drawings']
del cc_bal['DRAWINGS_POS']
del cc_bal['DRAWINGS_OTHER']
del cc_bal['DRAWINGS_ATM']
del cc_bal['DRAWINGS_CURRENT']

In [ ]:
#lets see the total number of times credit has been drawn
grp = cc_bal.groupby(by = ['SK_ID_CURR'])['CNT_DRAWINGS_ATM_CURRENT'].sum().reset_index().rename(index = str, columns = {'CNT_DRAWINGS_ATM_CURRENT' : 'ATM'})
cc_bal = cc_bal.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp
gc.collect()

grp = cc_bal.groupby(by = ['SK_ID_CURR'])['CNT_DRAWINGS_CURRENT'].sum().reset_index().rename(index = str, columns = {'CNT_DRAWINGS_CURRENT' : 'CURRENT'})
cc_bal = cc_bal.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp
gc.collect()

grp = cc_bal.groupby(by = ['SK_ID_CURR'])['CNT_DRAWINGS_OTHER_CURRENT'].sum().reset_index().rename(index = str, columns = {'CNT_DRAWINGS_OTHER_CURRENT' : 'OTHER'})
cc_bal = cc_bal.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp
gc.collect()

grp = cc_bal.groupby(by = ['SK_ID_CURR'])['CNT_DRAWINGS_POS_CURRENT'].sum().reset_index().rename(index = str, columns = {'CNT_DRAWINGS_POS_CURRENT' : 'POS'})
cc_bal = cc_bal.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp
gc.collect()

cc_bal['TOTAL_DRAWINGS'] = cc_bal['ATM'] + cc_bal['CURRENT'] + cc_bal['OTHER'] + cc_bal['POS']

del cc_bal['ATM']
del cc_bal['CURRENT']
del cc_bal['OTHER']
del cc_bal['POS']

In [ ]:
cc_bal.head()

In [ ]:
cc_bal['AMT_TOTAL_RECEIVABLE']

AMT_RECIVABLE and AMT_TOTAL_RECEIVABLE are basically the same columns . IF we subtract AMT_RECEIVABLE_PRINCIPAL from AMT_TOTAL_RECEIVABLE we will get the balance of bad debts for the customer in his previous credit. Might be an useful feature

In [ ]:
cc_bal.drop('AMT_RECIVABLE', axis=1, inplace=True)

In [ ]:
#Finding out the receivable amount from bad debts
grp = cc_bal.groupby(by = ['SK_ID_CURR'])['AMT_TOTAL_RECEIVABLE'].sum().reset_index().rename(index = str, columns = {'AMT_TOTAL_RECEIVABLE' : 'R_TOTAL'})
cc_bal = cc_bal.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp
gc.collect()

grp = cc_bal.groupby(by = ['SK_ID_CURR'])['AMT_RECEIVABLE_PRINCIPAL'].sum().reset_index().rename(index = str, columns = {'AMT_RECEIVABLE_PRINCIPAL' : 'R_PRIN'})
cc_bal = cc_bal.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp
gc.collect()

cc_bal['R_BAD_DEBTS'] = cc_bal['R_TOTAL'] - cc_bal['R_PRIN']
del cc_bal['R_TOTAL']
del cc_bal['R_PRIN']

In [ ]:
pv.head()

In [ ]:
grp = pv.groupby(by = ['SK_ID_CURR'])['AMT_ANNUITY'].sum().reset_index().rename(index = str, columns = {'AMT_ANNUITY' : 'ANNUITY_TOTAL'})
pv = pv.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp
gc.collect()

grp = pv.groupby(by = ['SK_ID_CURR'])['AMT_CREDIT'].sum().reset_index().rename(index = str, columns = {'AMT_CREDIT' : 'CREDIT_TOTAL'})
pv = pv.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp
gc.collect()

grp = pv.groupby(by = ['SK_ID_CURR'])['AMT_GOODS_PRICE'].sum().reset_index().rename(index = str, columns = {'AMT_GOODS_PRICE' : 'GP_TOTAL'})
pv = pv.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp
gc.collect()

pv['CREDIT_ANNUITY_RATIO'] =pv['CREDIT_TOTAL'] / pv['ANNUITY_TOTAL']
pv['ANNUITY_CREDIT_RATIO'] = pv['ANNUITY_TOTAL'] / pv['CREDIT_TOTAL']
pv['CREDIT_GP_RATIO'] = pv['CREDIT_TOTAL'] / pv['GP_TOTAL']

In [ ]:
app_train['new'] = np.where(app_train['FLAG_OWN_CAR']=='Y', 1, 0)
app_train['new1'] = np.where(app_train['FLAG_OWN_REALTY']=='Y', 1, 0)

In [ ]:
#application_test['IsSingle'] = np.where(application_test['NAME_FAMILY_STATUS'] == 'Single/not married', 1, 0)
application_train['RICH'] = np.where((application_train['FLAG_OWN_CAR']=='Y') & (application_train['FLAG_OWN_REALTY']=='Y'), 1, 0)

In [ ]:
application_train['>2child'] = np.where(application_train['CNT_CHILDREN'] >= 2, 1, 0)
application_train['>4fam'] = np.where(application_train['CNT_FAM_MEMBERS'] > 4, 1, 0)

In [ ]:
application_train['>4fam']

In [ ]:
plot_stats('RICH')

In [ ]:
application_train['bekar'] = np.where((application_train['NAME_FAMILY_STATUS']=='Civil marriage') & (application_train['CNT_CHILDREN'] >= 3), 1, 0)

In [ ]:
plot_stats('bekar')

In [ ]:
application_train['bekar1'] = np.where((application_train['NAME_FAMILY_STATUS']=='Married') & (application_train['OCCUPATION_TYPE']=='Low-skill Laborers'), 1, 0)

In [ ]:
plot_stats('bekar1')

In [ ]:
application_train['bekar1'] = np.where((application_train['NAME_FAMILY_STATUS']=='Married') & (application_train['NAME_INCOME_TYPE']=='Working'), 1, 0)

In [ ]:
plot_stats('bekar1')

In [ ]:
application_train['bekar1'] = np.where((application_train['NAME_FAMILY_STATUS']=='Married') & (application_train['NAME_INCOME_TYPE']=='Unemployed'), 1, 0)
#might be crucial

In [ ]:
plot_stats('bekar1')

In [ ]:
#We will now try out combinations of marriage status and housing type
application_train['bekar1'] = np.where((application_train['NAME_FAMILY_STATUS']=='Married') & 
                                       (application_train['NAME_HOUSING_TYPE']=='Rented apartment'), 1, 0)

In [ ]:
application_train['TOTAL_DOCS_SUBMITTED'] = application_train.loc[:, application_train.columns.str.contains('FLAG_DOCUMENT')].sum(axis=1)
plot_stats("TOTAL_DOCS_SUBMITTED")

In [ ]:
application_train['bekar1'] = np.where((application_train['NAME_FAMILY_STATUS']=='Married') & 
                                       (application_train['NAME_HOUSING_TYPE']=='House / apartment'), 1, 0)

In [ ]:
plot_stats('bekar1')

In [ ]:
application_train['bekar1'] = np.where((application_train['NAME_FAMILY_STATUS']=='Married') & 
                                       (application_train['NAME_HOUSING_TYPE']=='With parents'), 1, 0)

In [ ]:
plot_stats('bekar1')

In [ ]:
application_train['bekar1'] = np.where((application_train['NAME_FAMILY_STATUS']=='Civil marriage') & 
                                       (application_train['NAME_HOUSING_TYPE']=='Rented apartment'), 1, 0)

In [ ]:
plot_stats('bekar1')

In [ ]:
application_train['bekar1'] = np.where((application_train['NAME_FAMILY_STATUS']=='Civil marriage') & 
                                       (application_train['NAME_HOUSING_TYPE']=='With parents'), 1, 0)
plot_stats('bekar1')

In [ ]:
application_train['bekar1'] = np.where((application_train['NAME_FAMILY_STATUS']=='Civil marriage') & 
                                       (application_train['NAME_HOUSING_TYPE']=='House / apartment'), 1, 0)
plot_stats('bekar1')

In [ ]:
application_train['bekar1'] = np.where((application_train['NAME_INCOME_TYPE']=='Unemployed') & 
                                       (application_train['NAME_EDUCATION_TYPE']=='Secondary / secondary special'), 1, 0)
plot_stats('bekar1') #keep

In [ ]:
application_train['bekar1'] = np.where((application_train['NAME_INCOME_TYPE']=='Unemployed') & 
                                       (application_train['NAME_EDUCATION_TYPE']=='Incomplete higher'), 1, 0)
plot_stats('bekar1') #keep

In [ ]:
application_train['bekar1'] = np.where((application_train['NAME_INCOME_TYPE']=='Working') & 
                                       (application_train['NAME_EDUCATION_TYPE']=='Secondary / secondary special'), 1, 0)
plot_stats('bekar1')

In [ ]:
#Taking occupation type and education into account
application_train['bekar1'] = np.where((application_train['OCCUPATION_TYPE']=='Low-skill Laborers') & 
                                       (application_train['NAME_EDUCATION_TYPE']=='Incomplete higher'), 1, 0)
plot_stats('bekar1') #keep

In [ ]:
application_train['bekar1'] = np.where((application_train['OCCUPATION_TYPE']=='Low-skill Laborers') & 
                                       (application_train['NAME_EDUCATION_TYPE']=='Higher education'), 1, 0)
plot_stats('bekar1')

In [ ]:
application_train['bekar1'] = np.where((application_train['OCCUPATION_TYPE']=='Laborers') & 
                                       (application_train['NAME_EDUCATION_TYPE']=='Lower secondary'), 1, 0)
plot_stats('bekar1') #keep

In [ ]:
application_train['bekar1'] = np.where((application_train['OCCUPATION_TYPE']=='Laborers') & 
                                       (application_train['NAME_EDUCATION_TYPE']=='Secondary / secondary special'), 1, 0)
plot_stats('bekar1') #keep

In [ ]:
application_train['bekar1'] = np.where((application_train['OCCUPATION_TYPE']=='Drivers') & 
                                       (application_train['NAME_EDUCATION_TYPE']=='Lower secondary'), 1, 0)
plot_stats('bekar1')  #keep

In [ ]:
application_train['bekar1'] = np.where((application_train['OCCUPATION_TYPE']=='Drivers') & 
                                       (application_train['NAME_EDUCATION_TYPE']=='Secondary / secondary special'), 1, 0)
plot_stats('bekar1') #keep

In [ ]:
application_train['bekar1'] = np.where((application_train['OCCUPATION_TYPE']=='Low-skill Laborers') & 
                                       (application_train['NAME_FAMILY_STATUS']=='Married'), 1, 0)
plot_stats('bekar1') #keep

In [ ]:
application_train['bekar1'] = np.where((application_train['OCCUPATION_TYPE']=='Low-skill Laborers') & 
                                       (application_train['NAME_FAMILY_STATUS']=='Civil marriage'), 1, 0)
plot_stats('bekar1') #keep

In [ ]:
application_train['bekar1'] = np.where((application_train['OCCUPATION_TYPE']=='Low-skill Laborers') & 
                                       (application_train['NAME_FAMILY_STATUS']=='Single / not married'), 1, 0)
plot_stats('bekar1') #keep